In [1]:
# !pip3 install git+https://github.com/rongardF/tvdatafeed.git@e6f6aaa7de439ac6e454d9b26d2760ded8dc4923

from tvDatafeed import TvDatafeed, Interval

In [2]:
from helpers_db import get_engine, run_sql
engine = get_engine()

In [3]:
# queries
query_last_day = """SELECT DISTINCT a.symbol, a.exchange, count(t.dt)
                    FROM assets a LEFT JOIN daily_ticks t ON a.symbol = t.symbol
                    WHERE category = 'stock'
                    
                    GROUP BY a.symbol, a.exchange"""
                    # HAVING count(t.dt) = 0"""
                    # WHERE t.dt < (DATE_TRUNC('day', now()) - '1 day'::interval)
                    #  WHERE a.category = 'stock'
assets = None
with engine.begin() as conn:
  #  SELECT symbol, exchange FROM assets WHERE category='stock'
  assets = run_sql(conn, query_last_day).mappings().all()
# securities['symbol']
assets

[{'symbol': 'AAOI', 'exchange': 'NASDAQ', 'count': 171},
 {'symbol': 'AAPL', 'exchange': 'NASDAQ', 'count': 171},
 {'symbol': 'ADP', 'exchange': 'NASDAQ', 'count': 171},
 {'symbol': 'ALB', 'exchange': 'NYSE', 'count': 171},
 {'symbol': 'AMZN', 'exchange': 'NASDAQ', 'count': 171},
 {'symbol': 'BABA', 'exchange': 'NYSE', 'count': 171},
 {'symbol': 'CI', 'exchange': 'NYSE', 'count': 171},
 {'symbol': 'CLS', 'exchange': 'NYSE', 'count': 171},
 {'symbol': 'CRDF', 'exchange': 'NASDAQ', 'count': 171},
 {'symbol': 'CSCO', 'exchange': 'NASDAQ', 'count': 171},
 {'symbol': 'CVNA', 'exchange': 'NYSE', 'count': 171},
 {'symbol': 'DBK', 'exchange': 'XETR', 'count': 170},
 {'symbol': 'ESEA', 'exchange': 'NASDAQ', 'count': 171},
 {'symbol': 'EUR.USD', 'exchange': 'NASDAQ', 'count': 0},
 {'symbol': 'GCT', 'exchange': 'NASDAQ', 'count': 171},
 {'symbol': 'GOOG', 'exchange': 'NASDAQ', 'count': 171},
 {'symbol': 'HEI', 'exchange': 'NYSE', 'count': 171},
 {'symbol': 'INMD', 'exchange': 'NASDAQ', 'count': 1

In [4]:
from helpers_db import insert_on_conflict_nothing_ticks, truncate_table

tv = TvDatafeed()

def fetch_ticks(symbol, exchange, bars=100):
  df = tv.get_hist(
      symbol=symbol,
      exchange=exchange,
      interval=Interval.in_daily,
      n_bars=bars,
      extended_session=True,
  )
  return df

def fix_symbol(eachange_symbol):
  return eachange_symbol.split(':')[-1] if ':' in eachange_symbol else eachange_symbol

def normalize_dataframe(df):
  df.reset_index(inplace=True)
  df['symbol'] = df['symbol'].apply(fix_symbol)
  if 'datetime' in list(df.columns.values):
    df['dt'] = df['datetime']
    df.drop(columns=['datetime'], inplace=True)
  return df

# with engine.begin() as conn:
#   truncate_table(conn, 'ticks')

for asset in assets:
  symbol    = asset['symbol']
  exchange  = asset['exchange']
  print(f"Processing {exchange}:{symbol}")
  try:
    df = fetch_ticks(symbol, exchange, bars=7*24) # 7*24
    print(f"  count: {df.shape[0]}")
    df = normalize_dataframe(df)
    df.to_sql('daily_ticks', engine, if_exists='append', index=False, method=insert_on_conflict_nothing_ticks)
  except Exception as e:
    print(f"Exception {exchange}:{symbol}")
    print(str(e))


you are using nologin method, data you access may be limited


Processing NASDAQ:AAOI
  count: 168
Processing NASDAQ:AAPL
  count: 168
Processing NASDAQ:ADP
  count: 168
Processing NYSE:ALB
  count: 168
Processing NASDAQ:AMZN
  count: 168
Processing NYSE:BABA
  count: 168
Processing NYSE:CI
  count: 168
Processing NYSE:CLS
  count: 168
Processing NASDAQ:CRDF
  count: 168
Processing NASDAQ:CSCO
  count: 168
Processing NYSE:CVNA
  count: 168
Processing XETR:DBK
  count: 168
Processing NASDAQ:ESEA
  count: 168
Processing NASDAQ:EUR.USD


ERROR:tvDatafeed.main:Connection timed out
ERROR:tvDatafeed.main:no data, please check the exchange and symbol


Exception NASDAQ:EUR.USD
'NoneType' object has no attribute 'shape'
Processing NASDAQ:GCT
  count: 168
Processing NASDAQ:GOOG
  count: 168
Processing NYSE:HEI
  count: 168
Processing NASDAQ:INMD
  count: 168
Processing NYSE:JPM
  count: 168
Processing NYSE:KO
  count: 168
Processing EURONEXT:LQQ
  count: 168
Processing NASDAQ:META
  count: 168
Processing NASDAQ:MSFT
  count: 168
Processing NASDAQ:MSTR
  count: 168
Processing NASDAQ:NKTX
  count: 168
Processing NASDAQ:NVDA
  count: 168
Processing NASDAQ:ON
  count: 168
Processing NYSE:PAYC
  count: 168
Processing NASDAQ:PYPL
  count: 168
Processing NASDAQ:SBUX
  count: 168
Processing NASDAQ:SEDG
  count: 168
Processing NASDAQ:SURG
  count: 168
Processing NYSE:T
  count: 168
Processing NASDAQ:WDAY


ERROR:tvDatafeed.main:Connection to remote host was lost.
ERROR:tvDatafeed.main:no data, please check the exchange and symbol


Exception NASDAQ:WDAY
'NoneType' object has no attribute 'shape'
